In [1]:
import pandas as pd
import numpy as np
import ast

In [22]:
dataset = pd.read_csv(r'/mnt/bigDisk/weiren/Labels_TimeSyscallSeqs.csv')


In [35]:
count = dict()
for i in dataset['FAM']:
    if i in count.keys():
        count[i] += 1
    else:
        count[i] = 1
for i in count.keys():
    print(i," : ",count[i])



mirai  :  20475
unknown  :  8621
penguin  :  10
fakebank  :  47
fakeinst  :  21
mobidash  :  12
berbew  :  13
wroba  :  14
metasploit  :  5
zbot  :  1
skeeyah  :  1
hiddad  :  1


In [4]:
new_dataset = []
cnt_mirai = 0
cnt_unknown = 0
drop_fam = ['metasploit','zbot','skeeyah','hiddad']
drop_index = []
dataset = dataset.drop(['CLASS', 'BEH'], axis=1)
for i in range(len(dataset)):
    if dataset['FAM'][i] == 'mirai':
        if cnt_mirai > 50:
            drop_index.append(i)
        else:
            cnt_mirai += 1
    elif dataset['FAM'][i] == 'unknown':
        if cnt_unknown > 50 :
            drop_index.append(i)
        else :
            cnt_unknown += 1
    elif dataset['FAM'][i] in drop_fam:
        drop_index.append(i)
dataset = dataset.drop(drop_index)

In [5]:
dataset

,FILENAME,FAM,SEQUENCE
0,ca970cc75ccdf7bd1959b970de1f81d682ea3af871aff6...,mirai,"[('1659223215.715424', 'execve'), ('1659223215..."
1,699ffec8df22106524defd950bff835735051514a9b059...,mirai,"[('1662079445.156731', 'execve'), ('1662079445..."
2,94e883f37be5e6a2791b081e78c86804b427e90e55a172...,mirai,"[('946684902.916742', 'execve'), ('946684903.7..."
3,6b8490cc0ea6b234de8b6529b9277ee0ccba22026b4689...,unknown,"[('1970.5854504280915968', 'wait4'), ('1970.62..."
4,93a9669eea9b49258a1c4ca20b261561b6bd91b43643ad...,mirai,"[('1970.-1293549660274688', 'read'), ('1970.-1..."
...,...,...,...
27571,1baa017387e838d5cdf4e4cf3bbec00b593eb26a728536...,berbew,"[('1970.-2811361037910016', 'write'), ('1970.-..."
28136,86d7afdbc8ba2c20d95d48a350c46900d8d12186c8fea3...,mobidash,"[('1970.000000', 'execve'), ('2022.001355', 'o..."
28316,227f20d9a1f65144c9bd045ebcc38eac041ed56b7ba94b...,fakebank,"[('1970.000000', 'execve'), ('2022.008254', 'e..."
28636,3d243102bb07a0efe0a11d8ff3f9330d7161b29e151578...,fakebank,"[('1970.000000', 'execve'), ('2022.002493', 'r..."


In [6]:
# new_dataset = []
# cnt_mirai = 0
# cnt_unknown = 0
# drop_fam = ['metasploit','zbot','skeeyah','hiddad']
# drop_index = []
# dataset = dataset.drop(['CLASS', 'BEH'], axis=1)
# for i in range(len(dataset)):
#     # if dataset['FAM'][i] == 'mirai':
#     #     if cnt_mirai > 50:
#     #         drop_index.append(i)
#     #     else:
#     #         cnt_mirai += 1
#     # elif dataset['FAM'][i] == 'unknown':
#     #     if cnt_unknown > 50 :
#     #         drop_index.append(i)
#     #     else :
#     #         cnt_unknown += 1
#     if dataset['FAM'][i] in drop_fam:
#         drop_index.append(i)
# dataset = dataset.drop(drop_index)

In [15]:
dataset = dataset.reset_index(drop = True)
dataset["SEQUENCE"][0]

"[('1659223215.715424', 'execve'), ('1659223215.755751', 'rt_sigprocmask'), ('1659223215.768365', 'rt_sigaction'), ('1659223215.793790', 'rt_sigaction'), ('1659223215.804770', 'socket'), ('1659223215.818692', 'connect'), ('1659223215.820522', 'getsockname'), ('1659223215.839019', 'close'), ('1659223215.842809', 'brk'), ('1659223215.844117', 'brk'), ('1659223215.857712', 'time'), ('1659223215.860000"

In [9]:
syscall_list = []

for i in range(len(dataset)):
    tmp_syscall = []
    tmp = str(dataset["SEQUENCE"][i])
    tmp = tmp.replace('-','0')
    # print(i)
    tmp_seq = ast.literal_eval(tmp)
    end_t = float(tmp_seq[len(tmp_seq) - 1][0])
    start_t = float(tmp_seq[0][0])

    for j in range(len(tmp_seq)):
        try:
            time = float(tmp_seq[j][0])
            time -= start_t
            tmp_syscall.append([time,tmp_seq[j][1]])
        except:
            pass
    syscall_list.append(tmp_syscall)

In [17]:
syscallCategory = {
    'kernel_signal' : ['rt_sigaction', 'restart_syscall','kill', 'rt_sigprocmask','rt_sigsuspend','sigaltstack','tgkill','rt_sigtimedwait','sigreturn'],

    'kernel_sys' : ['uname', 'setgid', 'getpriority', 'sysinfo', 'prlimit64','geteuid','umask', 'getppid', 'setresuid','prctl', 'times','mmap','setsid','getuid', 'gettid', 'setpriority','setpgid', 'setresgid','getrlimit','getpid','setrlimit','getegid','setuid','getgid','getpgrp','mmap2','ni_syscall'],

    'kernel_others' : ['time','gettimeofday','alarm','clone','vfork','set_tid_address', 'fork','set_robust_list','futex','nanosleep','wait4','exit','exit_group','waitpid','ptrace','get_thread_area','set_thread_area','clock_gettime','setitimer'],

    'fs' : ['stat', 'lseek', 'readlinkat', 'chroot', 'sendfile', 'umount2', 'symlink', 'flock', 'dup2', 'getcwd', 'chdir', 'fstat', 'mount', 'rmdir', 'execve', 'mkdir', 'epoll_wait', 'openat', 'eventfd2', 'readv', 'rename', 'epoll_create1', 'fchmod', 'pipe', 'unlink', 'pipe2', 'fcntl', 'open', 'read', 'write', 'lstat', 'chmod', 'readlink', 'getdents64', 'utimes', 'ioctl', 'select', 'access', 'close', 'poll', 'getdents', 'epoll_ctl', 'ftruncate','_llseek','_newselect','fcntl64','fstat64','llseek','lstat64','renameat2','stat64'],

    'net' : ['accept', 'connect', 'sendto', 'shutdown', 'getsockname', 'getpeername', 'listen', 'socketpair', 'socket', 'setsockopt', 'getsockopt', 'recvfrom', 'recvmsg', 'bind','recv','send','sendfile64','socketcall'],

    'mm' : ['mprotect', 'brk', 'munmap', 'madvise'],

    'sched' : ['sched_getaffinity'],
    
    'HighFreq' : ['_newselect','close','connect','fcntl','get_thread_area','getsockopt','open','read','recv','recvfrom','rt_sigaction','rt_sigprocmask','sendto','socket','time'],

    'others' : [ 'getegid32','geteuid32','getgid32','getuid32','setgid32','setresuid32','setuid32','sysctl','ugetrlimit','syslog', 'shmdt', 'shmget']
}

In [11]:
fam = {
    'kernel_signal':0,
    'kernel_sys':1,
    'kernel_others':2,
    'fs':3,
    'net':4,
    'mm':5,
    'sched':6,
    'HighFreq':7,
    'others':8
}

In [12]:
def get_key(target):
    return_key = []
    for key,value in syscallCategory.items():
        if target in value:
            return_key.append(key)
            # return key
    if len(return_key) > 0:
        return return_key
    else:
        return ['others']

In [13]:
t_step = 16
column, row = t_step, len(syscallCategory)
imgs = []

for i in range(len(syscall_list)):
    time_step = np.linspace(0.0,float(syscall_list[i][-1][0]),num = t_step + 1)
    time_step = time_step[1:]
    k = 0
    step = [[0 for _ in range(row)] for _ in range(column)]
    for j in range(len(syscall_list[i])):
        if syscall_list[i][j][0] <= time_step[k]:
            syscall_fam = get_key(syscall_list[i][j][1])
            for h in syscall_fam:
                step[k][fam[h]] += 1
        else:
            k += 1
            syscall_fam = get_key(syscall_list[i][j][1])
            for h in syscall_fam:
                step[k][fam[h]] += 1                        
    imgs.append(step)


In [16]:
y = np.array(dataset["FAM"])
imgs = np.array(imgs)
np.save("./img_data_new.npy",imgs)
np.save("./img_label_new.npy",y)